# Lab 8: Recommender System

In this assignment, we will study how to do user-based collaborative filtering and item-based collaborative filtering. 

## 1. Dataset

In this assignment, we will use MovieLens-100K dataset. It includes about 100,000 ratings from 1000 users on 1700 movies.  

In [1]:
from math import sqrt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors


# 1. load data
user_ratings_train = pd.read_csv('./ml-100k/u1.base',
                            sep='\t',names=['user_id','movie_id','rating'], usecols=[0,1,2])

user_ratings_test = pd.read_csv('./ml-100k/u1.test',
                            sep='\t',names=['user_id','movie_id','rating'], usecols=[0,1,2])

movie_info =  pd.read_csv('./ml-100k/u.item', 
                          sep='|', names=['movie_id','title'], usecols=[0,1],
                          encoding="ISO-8859-1")

user_ratings_train = pd.merge(movie_info, user_ratings_train)
user_ratings_test = pd.merge(movie_info, user_ratings_test)

# 2. get the rating matrix. Each row is a user, and each column is a movie.
user_ratings_train = user_ratings_train.pivot_table(index=['user_id'],
                                        columns=['title'],
                                        values='rating')

user_ratings_test = user_ratings_test.pivot_table(index=['user_id'],
                                        columns=['title'],
                                        values='rating')




user_ratings_train = user_ratings_train.reindex(
                            index=user_ratings_train.index.union(user_ratings_test.index), 
                            columns=user_ratings_train.columns.union(user_ratings_test.columns) )

user_ratings_test = user_ratings_test.reindex(
                            index=user_ratings_train.index.union(user_ratings_test.index), 
                            columns=user_ratings_train.columns.union(user_ratings_test.columns) )

print(user_ratings_train.shape)
print(user_ratings_test.shape)

(943, 1664)
(943, 1664)


## Task 1. User-based CF

* Use pearson correlation to get the similarity between different users.
* Based on the obtained similarity score, predict the ratings. You can use 5 nearest neighbors or 10 nearest neighbors.
* Compute MAE for the testing set.

In [2]:
print(user_ratings_train)

title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
1                              NaN           NaN                    NaN   
2                              NaN           NaN                    NaN   
3                              NaN           NaN                    NaN   
4                              NaN           NaN                    NaN   
5                              NaN           NaN                    NaN   
...                            ...           ...                    ...   
939                            NaN           NaN                    NaN   
940                            NaN           NaN                    NaN   
941                            NaN           NaN                    NaN   
942                            NaN           NaN                    NaN   
943                            NaN           NaN                    NaN   

title    12 Angry Men (1

In [3]:
print(user_ratings_train.isnull().sum())

title
'Til There Was You (1997)                937
1-900 (1994)                             940
101 Dalmatians (1996)                    853
12 Angry Men (1957)                      841
187 (1997)                               905
                                        ... 
Young Guns II (1990)                     907
Young Poisoner's Handbook, The (1995)    910
Zeus and Roxanne (1997)                  939
unknown                                  940
Á köldum klaka (Cold Fever) (1994)       942
Length: 1664, dtype: int64


In [4]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_absolute_error
import math

#replace Nans with row avg
user_ratings_train['avg'] = user_ratings_train.mean(axis=1)
user_ratings_train_noNan = user_ratings_train.T.fillna(user_ratings_train['avg'], axis=0).T

pearson_sim_train = 1-pairwise_distances(user_ratings_train_noNan, metric="correlation")

train_model = NearestNeighbors(n_neighbors=10)
train_model.fit(pearson_sim_train)

neighbors_distance, neighbors_ind = train_model.kneighbors()
neighbors_ind += 1 # +1 fixes off by one error since ids start at 1 instead of 0

predictions = []
actual = []

for user_id, row in user_ratings_test.iterrows():
    for movie, rating in row.iteritems():
        if not pd.isnull(rating):
            predicted_rating = 0
            sum_of_sim = 0
            for x in range(0,10):
                ngbh_id = neighbors_ind[user_id-1][x]
                nghb_rating = user_ratings_train.loc[ngbh_id,movie]
                if not pd.isnull(nghb_rating):
                    nghb_distance = neighbors_distance[user_id-1][x]
                    sum_of_sim += nghb_distance
                    predicted_rating += nghb_distance*(nghb_rating-user_ratings_train.loc[ngbh_id, 'avg'])
            if (sum_of_sim != 0):
                predicted_rating = predicted_rating/sum_of_sim
                predicted_rating += user_ratings_train.loc[user_id, 'avg']
                predictions.append(predicted_rating)
                actual.append(rating)

mae = mean_absolute_error(predictions, actual)
print('MAE: ' + str(mae))

MAE: 0.8316716024705649


## Task 2. Item-based CF
* Use cosine similarity to get the similarity between different items.
* Based on the obtained similarity score, predict the ratings. You can use 5 nearest neighbors or 10 nearest neighbors.
* Compute MAE for the testing set.

In [5]:
item_ratings_train = user_ratings_train.T
item_ratings_test = user_ratings_test.T

#replace Nans with row avg
item_ratings_train['avg'] = item_ratings_train.mean(axis=1)
item_ratings_train_noNan = item_ratings_train.T.fillna(item_ratings_train['avg'], axis=0).T

#remove rows with no data
dropped_rows = item_ratings_train_noNan[item_ratings_train_noNan.isna().any(axis=1)]
item_ratings_train_noNan = item_ratings_train_noNan.drop(dropped_rows.index)
item_ratings_test = item_ratings_test.drop(dropped_rows.index)

pearson_sim_train = 1-pairwise_distances(item_ratings_train_noNan, metric="cosine")

train_model = NearestNeighbors(n_neighbors=10)
train_model.fit(pearson_sim_train)

neighbors_distance, neighbors_ind = train_model.kneighbors()
neighbors_ind += 1 # +1 fixes off by one error since ids start at 1 instead of 0

predictions = []
actual = []

for movie_id, row in item_ratings_test.iterrows():
    item_id = item_ratings_test.index.get_loc(movie_id)
    for user, rating in row.iteritems():
        if not pd.isnull(rating):
            predicted_rating = 0
            sum_of_sim = 0
            for x in range(0,10):
                ngbh_id = neighbors_ind[item_id][x]
                nghb_rating = item_ratings_train.iloc[ngbh_id].loc[user]
                if not pd.isnull(nghb_rating):
                    nghb_distance = neighbors_distance[item_id][x]
                    sum_of_sim += nghb_distance
                    predicted_rating += nghb_distance*(nghb_rating)
            if (sum_of_sim != 0):
                predicted_rating = predicted_rating/sum_of_sim
                predictions.append(predicted_rating)
                actual.append(rating)

mae = mean_absolute_error(predictions, actual)
print('MAE: ' + str(mae))

MAE: 1.0373153924470926
